In [13]:
from pymongo import MongoClient
from sshtunnel import SSHTunnelForwarder

class MongoConnectionHandlerException(Exception):
    pass

class MongoConnectionHandler:

    __localhost = '127.0.0.1'

    def __init__(self):

        server = SSHTunnelForwarder(
            ('18.130.129.216', 22),
            ssh_username='mongo',
            ssh_password='7YPbhiYTEtM=',
            remote_bind_address=('192.168.2.91', 27017)
        )
        server.start()

        self.__client = MongoClient(host=self.__localhost, 
                                    port=server.local_bind_port, 
                                    username='mongo-admin', 
                                    password='7YPbhiYTEtM=', 
                                    retryWrites=False)

    @property
    def client(self):
        if self.__client is None:
            raise MongoConnectionHandlerException('Invalid database connection.')

        return self.__client

    def close(self):
        self.__client.close()

class MongoRepository:
    
    def __init__(self, client):
        
        self.__client = client
        self.__collection = None
        
    def add_one(self, document):
        if not document:
            return
        try:
            self.__client['dev_facebook_turing_insights_mock'][self.__collection].insert_one(document)
        except Exception as e:
            raise e
            
    def get_structures_with_interests(self):
        return list(self.__client.dev_facebook_turing_structures.adset.find({}))
    
    def decode_stuff(self):
        from bson import BSON
        structures = self.get_structures_with_interests()
        for structure in structures:
            ceva = BSON.decode(structure.get("details", {}))
            print(ceva)
        
    def set_collection(self, collection, breakdown):
        self.__collection = collection.value + '_' + breakdown.value + '_none'
            

In [14]:
from enum import Enum

class LevelEnum(Enum):
    
    AD = 'ad'
    ADSET = 'adset'
    CAMPAIGN = 'campaign'
    
class ObjectiveEnum(Enum):
    APP_INSTALLS = 'APP_INSTALLS'
    BRAND_AWARENESS = 'BRAND_AWARENESS'
    CONVERSIONS = 'CONVERSIONS'
    EVENT_RESPONSES = 'EVENT_RESPONSES'
    LEAD_GENERATION = 'LEAD_GENERATION'
    LINK_CLICKS = 'LINK_CLICKS'
    MESSAGES = 'MESSAGES'
    PAGE_LIKES = 'PAGE_LIKES'
    POST_ENGAGEMENT = 'POST_ENGAGEMENT'
    PRODUCT_CATALOG_SALES = 'PRODUCT_CATALOG_SALES'
    REACH = 'REACH'
    VIDEO_VIEWS = 'VIDEO_VIEWS'

class BreakdownEnum(Enum):
    NONE = "none"
    AGE = "age_breakdown"
    GENDER = "gender_breakdown"
    PLACEMENT = "placement"
    DEVICE = "impression_device"
    PLATFORM = "publisher_platform"
    
mongo_client = MongoConnectionHandler()

In [15]:
import numpy
import random

def date_plus_one(date):
    new_date = []
    
    split_date = date.split("-")
    new_date.append(split_date[0])
    next_day = str(int(split_date[2]) + 1)
    if next_day == '31':
        split_date[1] = '05'
        next_day = '01'
    if len(next_day) != 2:
        next_day = '0' + next_day
    
    split_date[2] = next_day
    
    new_date.append(split_date[1])
    new_date.append(split_date[2])
    
    return "-".join(new_date)

def document_creator_base(account_id=None, objective=None, campaign_id=None, mongo_repo=None, adset_id=None, ad_id=None, breakdown=None, date=None):
    initial_obj = {
    "date_stop" : None, # "2020-05-06"
    "date_start" : None,
    "ad_name" : 'MOCK',
    "ad_id" : ad_id,
    "adset_name" : 'MOCK',
    "adset_id" : adset_id,
    "campaign_name" : "MOCK",
    "campaign_id" : campaign_id,
    "account_name" : "MOCK",
    "account_id" : account_id,
    "quality_ranking" : "UNKNOWN",
    "objective" : objective.value,
    "clicks_all" : None,
    "purchases_value" : None,
    "post_engagement" : None,
    "photo_views" : None,
    "post_shares" : None,
    "post_comments" : None,
    "unique_link_clicks" : None,
    "thru_plays" : None,
    "conversions" : None,
    "website_app_installs_total" : None,
    "event_responses" : None,
    "leads_total" : None,
    "video_plays" : None,
    "purchases_total" : None,
    "impressions" : None,
    "page_likes" : None,
    "link_clicks" : None,
    "reach" : None,
    "amount_spent" : None,
    "results" : None
    }
    
    ONE_TO_THOUSAND = numpy.arange(0, 1000, 1)
    ONE_TO_FIFTY = numpy.arange(0, 50, 1)
    ONE_TO_HUNDRED = numpy.arange(0, 100, 1)
    
    initial_obj = dict()
    initial_obj["ad_id"] = ad_id
    initial_obj["adset_name"] = 'MOCK'
    initial_obj["adset_id"] = adset_id
    initial_obj["campaign_name"] = "MOCK"
    initial_obj["campaign_id"] = campaign_id
    initial_obj["account_name"] = "MOCK"
    initial_obj["account_id"] = account_id
    initial_obj["quality_ranking"] = "UNKNOWN"
    initial_obj["objective"] : objective.value
    initial_obj['date_start'] = date
    initial_obj['date_stop'] = date
    initial_obj["clicks_all"] = int(random.choice(ONE_TO_THOUSAND))
    initial_obj["purchases_value"] = int(random.choice(ONE_TO_THOUSAND))
    initial_obj["post_engagement"] = int(random.choice(ONE_TO_FIFTY))
    initial_obj["photo_views"] = int(random.choice(ONE_TO_FIFTY))
    initial_obj["post_shares"] = int(random.choice(ONE_TO_FIFTY))
    initial_obj["post_comments"] = int(random.choice(ONE_TO_FIFTY))
    initial_obj["unique_link_clicks"] = int(random.choice(ONE_TO_THOUSAND))
    initial_obj["thru_plays"] = int(random.choice(ONE_TO_FIFTY))
    initial_obj["conversions"] = int(random.choice(numpy.arange(0, 5000, 1)))
    initial_obj["website_app_installs_total"] = int(random.choice(ONE_TO_THOUSAND))
    initial_obj["event_responses"] = int(random.choice(ONE_TO_FIFTY))
    initial_obj["leads_total"] = int(random.choice(ONE_TO_FIFTY))
    initial_obj["video_plays"] = int(random.choice(ONE_TO_FIFTY))
    initial_obj["purchases_total"] = int(random.choice(ONE_TO_HUNDRED))
    initial_obj["impressions"] = int(random.choice(ONE_TO_THOUSAND))
    initial_obj["page_likes"] = int(random.choice(ONE_TO_HUNDRED))
    initial_obj["link_clicks"] = int(random.choice(ONE_TO_FIFTY))
    initial_obj["reach"] = int(random.choice(numpy.arange(0, 10000, 1)))
    initial_obj["amount_spent"] = float(random.choice(numpy.arange(0, 1000, 0.89)))
    initial_obj["results"] = int(random.choice(ONE_TO_FIFTY))
    
    return initial_obj
        
def document_creator(account_id=None, objective=None, campaign_id=None, mongo_repo=None, adset_id=None, ad_id=None, breakdown=None):   
    date = '2020-04-01'
    final_list = []
    for _ in range(1, 60):
        if breakdown == BreakdownEnum.AGE:
            for breakdown_value in ['18-25', '35-45', '45-55', '55-65', '65+']:
                data = document_creator_base(account_id=account_id, objective=objective, campaign_id=campaign_id, 
                                             mongo_repo=mongo_repo, adset_id=adset_id, ad_id=ad_id, 
                                             breakdown=breakdown, date=date)
                data[breakdown.value] = breakdown_value
                final_list.append(data.copy())
        elif breakdown == BreakdownEnum.GENDER:
            for breakdown_value in ['male', 'female', 'unknown']:
                data = document_creator_base(account_id=account_id, objective=objective, campaign_id=campaign_id, 
                                             mongo_repo=mongo_repo, adset_id=adset_id, ad_id=ad_id, 
                                             breakdown=breakdown, date=date)
                data[breakdown.value] = breakdown_value
                final_list.append(data.copy())
        elif breakdown == BreakdownEnum.PLACEMENT:
            for breakdown_value in ['audience_network,an_classic,desktop',
                                    'audience_network,an_classic,mobile',
                                    'facebook,facebook_stories,mobile',
                                    'facebook,feed,desktop']:
                data = document_creator_base(account_id=account_id, objective=objective, campaign_id=campaign_id, 
                                             mongo_repo=mongo_repo, adset_id=adset_id, ad_id=ad_id, 
                                             breakdown=breakdown, date=date)
                data[breakdown.value] = breakdown_value
                final_list.append(data.copy())
        elif breakdown == BreakdownEnum.DEVICE:
            for breakdown_value in ['android_smartphone', 'android_tablet', 
                                    'desktop', 'ipad', 'iphone', 'ipod', 'other']:
                data = document_creator_base(account_id=account_id, objective=objective, campaign_id=campaign_id, 
                                             mongo_repo=mongo_repo, adset_id=adset_id, ad_id=ad_id, 
                                             breakdown=breakdown, date=date)
                data[breakdown.value] = breakdown_value
                final_list.append(data.copy())
        elif breakdown == BreakdownEnum.PLATFORM:
            for breakdown_value in ['audience_network', 'facebook', 'instagram', 'messenger']:
                data = document_creator_base(account_id=account_id, objective=objective, campaign_id=campaign_id, 
                                             mongo_repo=mongo_repo, adset_id=adset_id, ad_id=ad_id, 
                                             breakdown=breakdown, date=date)
                data[breakdown.value] = breakdown_value
                final_list.append(data.copy())
        else:
            data = document_creator_base(account_id=account_id, objective=objective, campaign_id=campaign_id, 
                                         mongo_repo=mongo_repo, adset_id=adset_id, ad_id=ad_id, 
                                         breakdown=breakdown, date=date)
            final_list.append(data)
        
        date = date_plus_one(date)

    for element in final_list:
        mongo_repo.add_one(element)


In [16]:
business_owner_facebook_id = '1623950661230875'
ids = [
    {
        "business_owner_facebook_id": "1623950661230875",
        "account_id": "2242745292656390",
        "campaigns": [
            {
                "id": "23844372658000216",
                 "children": [
                     {
                         "id": "23844379502300216",
                         "children": [
                             "23844379502310216",
                             "23844379502240216",
                             "23844379502230216",
                             "23844379502160216"
                         ]
                     }
                 ]
            }
        ]
    }
]

In [17]:
mongo_repo = MongoRepository(mongo_client.client)
# mongo_repo.decode_stuff()

for objective in ObjectiveEnum:
    for breakdown in BreakdownEnum:
        for entry in ids:
            for campaign in entry["campaigns"]:
                # Set the collection to be 'campaign-breakdown-none'
                mongo_repo.set_collection(LevelEnum.CAMPAIGN, breakdown=breakdown)

                # Create the documents in the collection
                document_creator(account_id=entry["account_id"], 
                                 objective=objective, 
                                 campaign_id=campaign["id"],
                                 mongo_repo=mongo_repo,
                                 breakdown=breakdown)
                for adset in campaign["children"]:
                    # Set the collection to be 'adset-breakdown-none'
                    mongo_repo.set_collection(LevelEnum.ADSET, breakdown=breakdown)

                    # Create the documents in the collection
                    document_creator(account_id=entry["account_id"], 
                                     objective=objective, 
                                     campaign_id=campaign["id"],
                                     adset_id=adset["id"],
                                     mongo_repo=mongo_repo,
                                     breakdown=breakdown)
                    for ad_id in adset["children"]:
                        # Set the collection to be 'adset-breakdown-none'
                        mongo_repo.set_collection(LevelEnum.AD, breakdown=breakdown)

                        # Create the documents in the collection
                        document_creator(account_id=entry["account_id"], 
                                         objective=objective, 
                                         campaign_id=campaign["id"],
                                         adset_id=adset["id"],
                                         ad_id=ad_id,
                                         mongo_repo=mongo_repo,
                                         breakdown=breakdown)